## 크롤링에 필요한 라이브러리

In [2]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup as bs

In [ ]:
driver = webdriver.Chrome("C:/Program Files/Google/Chrome/Application/chrome.exe")


C:\Users\718\AppData\Local\Temp/ipykernel_7628/997313917.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("C:/Program Files/Google/Chrome/Application/chrome.exe")


In [ ]:
driver.get("https://cafe.naver.com/mbticafe")
driver.implicitly_wait(3)

## 네이버 로그인

In [4]:
# # 로그인 전용 화면
# driver.get('https://nid.naver.com/nidlogin.login')
# # 아이디와 비밀번호 입력
# driver.find_element_by_name('id').send_keys('아이디')
# driver.find_element_by_name('pw').send_keys('비밀번호')
# # 로그인 버튼 클릭
# driver.find_element_by_css_selector('#frmNIDLogin > fieldset > input').click()

## 게시판 목록으로 접근

In [5]:
base_url = 'https://cafe.naver.com/mbticafe/ArticleList.nhn?search.clubid=11856775&amp'
driver.get(base_url + '&search.menuid=18&amp&search.page=***')
# iframe으로 프레임 전환
driver.switch_to.frame('cafe_main')

In [7]:
<a href="/ArticleList.nhn?search.clubid=11856775&amp;search.menuid=18&amp;search.boardtype=L" 
target="cafe_main" onclick="goMenu('18');clickcr(this, 'mnu.normal','','',event);" 
class="gm-tcol-c b" id="menuLink18">
						INFP ♧ ENFP
					</a>

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 5)

In [ ]:
# href 속성을 찾아 url을 리스트로 저장한다.
## 각 카페마다 태그의 형식이 다를 수 있으니 확인해야 한다. ( 태그.클래스명 )
article_list = driver.find_elements_by_css_selector('span.aaa > a.m-tcol-c')
article_urls = [ i.get_attribute('href') for i in article_list ]

## 게시글로 접근

In [ ]:
res_list = []
# Beautifulsoup 활용
for article in article_urls:
    driver.get(article)
    # article도 switch_to.frame이 필수
    driver.switch_to.frame('cafe_main')
    soup = bs(driver.page_source, 'html.parser')
    # 게시글에서 제목 추출
    title = soup.select('div.tit-box span.b')[0].get_text()
    # 내용을 하나의 텍스트로 만든다. (띄어쓰기 단위)
    content_tags = soup.select('#tbody')[0].select('p')
    content = ' '.join([ tags.get_text() for tags in content_tags ])
    # dict형태로 만들어 결과 list에 저장
    res_list.append({'title' : title, 'content' : content})
    # time.sleep 작업도 필요하다.
# 결과 데이터프레임화
cafe_df = pd.DataFrame(res_list)
# csv파일로 추출
cafe_df.to_csv('cafe_crawling.csv', index=False)